In [1]:
import pandas as pd

from utils import get_file_from_gsheets

In [2]:
FILE_NAME = "input-data"

In [3]:
pd.options.display.max_columns = None

pd.options.display.max_rows = None

# Выгрузка исходных данных и преведение форматов

note: Перенес таблицы из ворда в google sheets

### Первая таблица

In [4]:
sku_list_one = get_file_from_gsheets(file_name=FILE_NAME, sheet_name='sheet_1')

sku_list_one.shape

(8, 3)

In [5]:
sku_list_one.columns = sku_list_one.columns.str.lower()

In [6]:
sku_list_one =\
    sku_list_one\
        .rename(columns={
            "код скю1": "sku_id_1",
        })

In [7]:
sku_list_one\
    .duplicated(subset='sku_id_1', keep=False)\
    .sum()

0

### Вторая таблица

In [8]:
sku_list_two = get_file_from_gsheets(file_name=FILE_NAME, sheet_name='sheet_2')

sku_list_two.shape

(10, 3)

In [9]:
sku_list_two.columns = sku_list_two.columns.str.lower()

In [10]:
sku_list_two =\
    sku_list_two\
        .rename(columns={
            "код скю1": "sku_id_1",
        })

### Третья таблица

In [11]:
sku_list_three = get_file_from_gsheets(file_name=FILE_NAME, sheet_name='sheet_3')

sku_list_three.shape

(5, 4)

In [13]:
sku_list_three.columns = sku_list_three.columns.str.lower()

sku_list_three = sku_list_three.rename(columns={
    'канал':  'channel',
    'код скю1': 'sku_id_1',
    'name': 'category'
})

### Справочник канал-подканал

In [46]:
sub_channels_reference = get_file_from_gsheets(file_name=FILE_NAME, sheet_name='sheet_4')

sub_channels_reference.shape

(39, 2)

In [47]:
data = sub_channels_reference.columns
sub_channels_reference.iloc[-1] = data

cols = ['channel', 'sub_channel']
sub_channels_reference.columns = cols

In [48]:
sub_channels_reference.head()

,channel,sub_channel
0,FT,542
1,FT,810
2,FT,107
3,FT,108
4,FT,111


### Матринг sku_1 к sku_2

In [37]:
sku_mathing = get_file_from_gsheets(file_name=FILE_NAME, sheet_name='sheet_5')

sku_mathing.shape

(22, 2)

In [39]:
sku_mathing.columns = sku_mathing.columns.str.lower()

In [40]:
sku_mathing = sku_mathing.rename(
    columns={
        'код скю1': 'sku_id_1',
        'код скю2': 'sku_id_2'
    }
)

# Матчинг списков SKU и их под-каналов

In [58]:
sub_channels_list = [150, 154]

df = pd.DataFrame(data=sub_channels_list, columns=['sub_channel'])

df = df.merge(sku_list_one[['sku_id_1', 'rank']], how='cross')

In [60]:
sub_channels_list = [255, 799, 798]

df2 = pd.DataFrame(data=sub_channels_list, columns=['sub_channel'])

df2 = df2.merge(sku_list_two[['sku_id_1', 'rank']], how='cross')

In [61]:
sub_channels_reference = sub_channels_reference[sub_channels_reference['channel']=='FT']

In [62]:
sub_channels_reference.duplicated(subset=['sub_channel'], keep=False).sum()

0

In [64]:
df3 = sub_channels_reference[['sub_channel']].merge(right=sku_list_three[['sku_id_1', 'rank']], how='cross')

# Создание финального файла

In [67]:
concated_df = pd.concat(objs=[df, df2, df3]).reset_index().drop(columns='index')

concated_df.shape

(126, 3)

In [71]:
result_df = concated_df.merge(right=sku_mathing, how='left', on=['sku_id_1'])

In [73]:
result_df = result_df[['sub_channel', 'sku_id_2', 'rank']]

In [76]:
# Проверка на дубли связки подканал - SKU

assert result_df.duplicated(subset=['sub_channel', 'sku_id_2'], keep=False).sum() == 0

In [78]:
result_df.shape

(269, 3)

In [77]:
result_df.head(50)

,sub_channel,sku_id_2,rank
0,150,111,4
1,150,999,4
2,150,222,5
3,150,1110,5
4,150,2220,5
5,150,333,1
6,150,1919,1
7,150,2331,1
8,150,444,6
9,150,555,2
